## train the RNN model

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

features_df = pd.read_csv('features.csv')

# Extract features and corresponding labels
X = features_df.iloc[:, :-1].values  # features 
y = features_df.iloc[:, -1].values  # labels 

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encode the labels
y = label_encoder.fit_transform(y)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = X_train.shape[1]
hidden_size = 64 
num_classes = len(np.unique(y_train))

model = RNN(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300 

for epoch in range(num_epochs):
    model.train()
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    outputs = model(X_train_tensor.unsqueeze(1))  
    optimizer.zero_grad()
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    # Validation
    model.eval()
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    with torch.no_grad():
        val_outputs = model(X_val_tensor.unsqueeze(1))
        val_loss = criterion(val_outputs.squeeze(), y_val)
        _, val_predicted = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_val.numpy(), val_predicted.numpy())

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Evaluate the model on the test set
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(X_test_tensor.unsqueeze(1))
    _, test_predicted = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test.numpy(), test_predicted.numpy())

print("Test Accuracy:", test_accuracy)

Epoch [1/300], Training Loss: 0.5428
Epoch [1/300], Validation Loss: 0.4828, Validation Accuracy: 0.9159
Epoch [2/300], Training Loss: 0.4791
Epoch [2/300], Validation Loss: 0.4269, Validation Accuracy: 0.9373
Epoch [3/300], Training Loss: 0.4230
Epoch [3/300], Validation Loss: 0.3782, Validation Accuracy: 0.9537
Epoch [4/300], Training Loss: 0.3743
Epoch [4/300], Validation Loss: 0.3360, Validation Accuracy: 0.9615
Epoch [5/300], Training Loss: 0.3321
Epoch [5/300], Validation Loss: 0.2996, Validation Accuracy: 0.9636
Epoch [6/300], Training Loss: 0.2957
Epoch [6/300], Validation Loss: 0.2682, Validation Accuracy: 0.9658
Epoch [7/300], Training Loss: 0.2645
Epoch [7/300], Validation Loss: 0.2412, Validation Accuracy: 0.9701
Epoch [8/300], Training Loss: 0.2377
Epoch [8/300], Validation Loss: 0.2180, Validation Accuracy: 0.9701
Epoch [9/300], Training Loss: 0.2147
Epoch [9/300], Validation Loss: 0.1980, Validation Accuracy: 0.9701
Epoch [10/300], Training Loss: 0.1949
Epoch [10/300], V

Epoch [81/300], Training Loss: 0.0266
Epoch [81/300], Validation Loss: 0.0267, Validation Accuracy: 0.9907
Epoch [82/300], Training Loss: 0.0264
Epoch [82/300], Validation Loss: 0.0264, Validation Accuracy: 0.9907
Epoch [83/300], Training Loss: 0.0262
Epoch [83/300], Validation Loss: 0.0261, Validation Accuracy: 0.9907
Epoch [84/300], Training Loss: 0.0259
Epoch [84/300], Validation Loss: 0.0259, Validation Accuracy: 0.9914
Epoch [85/300], Training Loss: 0.0257
Epoch [85/300], Validation Loss: 0.0256, Validation Accuracy: 0.9914
Epoch [86/300], Training Loss: 0.0255
Epoch [86/300], Validation Loss: 0.0253, Validation Accuracy: 0.9914
Epoch [87/300], Training Loss: 0.0252
Epoch [87/300], Validation Loss: 0.0251, Validation Accuracy: 0.9914
Epoch [88/300], Training Loss: 0.0250
Epoch [88/300], Validation Loss: 0.0248, Validation Accuracy: 0.9922
Epoch [89/300], Training Loss: 0.0248
Epoch [89/300], Validation Loss: 0.0245, Validation Accuracy: 0.9922
Epoch [90/300], Training Loss: 0.0246

Epoch [168/300], Training Loss: 0.0140
Epoch [168/300], Validation Loss: 0.0123, Validation Accuracy: 0.9964
Epoch [169/300], Training Loss: 0.0139
Epoch [169/300], Validation Loss: 0.0122, Validation Accuracy: 0.9964
Epoch [170/300], Training Loss: 0.0138
Epoch [170/300], Validation Loss: 0.0121, Validation Accuracy: 0.9964
Epoch [171/300], Training Loss: 0.0137
Epoch [171/300], Validation Loss: 0.0120, Validation Accuracy: 0.9964
Epoch [172/300], Training Loss: 0.0137
Epoch [172/300], Validation Loss: 0.0120, Validation Accuracy: 0.9964
Epoch [173/300], Training Loss: 0.0136
Epoch [173/300], Validation Loss: 0.0119, Validation Accuracy: 0.9964
Epoch [174/300], Training Loss: 0.0135
Epoch [174/300], Validation Loss: 0.0118, Validation Accuracy: 0.9979
Epoch [175/300], Training Loss: 0.0134
Epoch [175/300], Validation Loss: 0.0117, Validation Accuracy: 0.9979
Epoch [176/300], Training Loss: 0.0133
Epoch [176/300], Validation Loss: 0.0116, Validation Accuracy: 0.9979
Epoch [177/300], Tr

Epoch [255/300], Training Loss: 0.0084
Epoch [255/300], Validation Loss: 0.0069, Validation Accuracy: 1.0000
Epoch [256/300], Training Loss: 0.0084
Epoch [256/300], Validation Loss: 0.0069, Validation Accuracy: 1.0000
Epoch [257/300], Training Loss: 0.0083
Epoch [257/300], Validation Loss: 0.0068, Validation Accuracy: 1.0000
Epoch [258/300], Training Loss: 0.0083
Epoch [258/300], Validation Loss: 0.0068, Validation Accuracy: 1.0000
Epoch [259/300], Training Loss: 0.0082
Epoch [259/300], Validation Loss: 0.0068, Validation Accuracy: 1.0000
Epoch [260/300], Training Loss: 0.0082
Epoch [260/300], Validation Loss: 0.0067, Validation Accuracy: 1.0000
Epoch [261/300], Training Loss: 0.0081
Epoch [261/300], Validation Loss: 0.0067, Validation Accuracy: 1.0000
Epoch [262/300], Training Loss: 0.0081
Epoch [262/300], Validation Loss: 0.0066, Validation Accuracy: 1.0000
Epoch [263/300], Training Loss: 0.0081
Epoch [263/300], Validation Loss: 0.0066, Validation Accuracy: 1.0000
Epoch [264/300], Tr

## Test phase 

In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

# Specify the directories containing the .mp3 files
directories = ['something']

# Create an empty DataFrame 
features_df = pd.DataFrame()

for directory in directories:
    print(f"Processing files in {directory} directory")
    for filename in tqdm(os.listdir(directory)):
        if filename.endswith('.wav'):
            file_path = os.path.join(directory, filename)
            try:
                features = extract_features(file_path)
                # Append the features to the DataFrame as a new row
                if features is not None:
                    features_series = pd.Series(features)
                    features_df = pd.concat([features_df, features_series], axis=0)  # Concatenate along rows (axis=0)
            except Exception as e:
                print(f"Error encountered while processing file: {file_path}")
                continue


Processing files in something directory


100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.32s/it]


In [3]:
X_new= features_df.T
X_new.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,-224.141861,123.548615,-11.934964,35.14872,-2.978472,11.988633,-10.192609,7.214376,-13.9877,1.976113,...,20.868756,15.711451,19.730115,19.38868,19.780333,19.137458,41.547228,1692.69833,0.082245,3374.557483


## load the model

In [4]:

loaded_model = RNN( input_size, hidden_size, num_classes)


loaded_model.load_state_dict(torch.load('rnn_model.pth'))


<All keys matched successfully>

In [5]:
loaded_model.eval()  


X_new = scaler.transform(X_new) 
X_new_tensor = torch.tensor(X_new, dtype=torch.float32)

with torch.no_grad():
    new_outputs = loaded_model(X_new_tensor.unsqueeze(1))
    _, new_predicted = torch.max(new_outputs, 1)




In [6]:
print(new_predicted)

tensor([1])


In [19]:
import pyaudio
import wave
import time
import os
import pandas as pd  # Import pandas if not already imported
import torch  # Import PyTorch if not already imported


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    ten_sec_frames = int(sample_rate / 1024 * 10)  # Number of frames for 10 seconds

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        # Slide the window by 10 seconds
        print("Sliding the window by 10 seconds...")
        audio_frames = audio_frames[ten_sec_frames:]  # Remove the first 10 seconds
        
        # Record new 10 seconds of audio data
        for _ in range(0, ten_sec_frames):
            data = stream.read(1024)
            audio_frames.append(data)

        # Save the recorded audio to a WAV file
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(3)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()


Initializing audio buffer with 1-minute audio...
Initialization complete.
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the window by 10 seconds...
Predicted Class: 1.0
Sliding the wi

KeyboardInterrupt: 

In [ ]:
import cv2

def test_video_capture():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error: Couldn't open the camera.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read a frame.")
            break

        cv2.imshow('Test', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

test_video_capture()


In [12]:
import pyaudio
import wave
import time
import os
import pandas as pd  
import torch


def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None

def record_and_classify_audio():
    audio_duration = 60  # Record audio for 1 minute
    sample_rate = 44100  # 44.1kHz
    _frames_ = int(sample_rate / 1024 * 0.1)  

    audio_frames = []
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)
    
    # Initialize the buffer with 1-minute audio data
    print("Initializing audio buffer with 1-minute audio...")
    for _ in range(0, int(sample_rate / 1024 * audio_duration)):
        data = stream.read(1024)
        audio_frames.append(data)
    print("Initialization complete.")

    while True:
        
        print("Sliding the window by 0.1 seconds...")
        audio_frames = audio_frames[_frames_:]
        
        for _ in range(0, _frames_):
            data = stream.read(1024)
            audio_frames.append(data)

        
        output_audio_file = "recorded_audio.wav"
        wf = wave.open(output_audio_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(audio_frames))
        wf.close()

        # extract features 
        recorded_features = extract_features(output_audio_file)

        if recorded_features is not None:
            recorded_df = pd.DataFrame([recorded_features])
            recorded_df = scaler.transform(recorded_df)
            model.eval()
            recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
            with torch.no_grad():
                predicted_output = model(recorded_tensor.unsqueeze(1))
                _, predicted_class = torch.max(predicted_output, 1)

            predicted_class_label = label_encoder.inverse_transform(predicted_class.numpy())
            print("Predicted Class:", predicted_class_label[0])

            # Delete the audio file
            os.remove(output_audio_file)
        else:
            print("Error occurred while extracting features from recorded audio.")

        time.sleep(0.1)

# Entry point
if __name__ == "__main__":
    record_and_classify_audio()

Initializing audio buffer with 1-minute audio...
Initialization complete.
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.

Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Clas

Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Clas

Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Clas

Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Clas

Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Class: 1.0
Sliding the window by 0.1 seconds...
Predicted Clas

KeyboardInterrupt: 